## LCNN


In [1]:
import os
import pandas as pd
import numpy as np
import scipy as sp
import scipy.stats
import librosa
import librosa.display
import math
import sys
#sys.path.insert(0,'/home/ikwak2/hmd/notebooks')
#sys.path.insert(0,'/home/ikwak2/hmd/iy_classifier')
sys.path.insert(0,'utils')
from helper_code import *
from get_feature import *
from models import *
from Generator0 import *

In [2]:
root_dir = 'physionet.org/files/circor-heart-sound/1.0.3'
training_data_file = root_dir + '/' + 'training_data.csv'
training_data_dir = root_dir + '/' + 'training_data'
model_dir = root_dir + '/' + 'model'

In [3]:
import tensorflow as tf

gpu_info = !nvidia-smi
gpu_info = '\n'.join(gpu_info)
print(gpu_info)

Fri Jul 15 10:34:52 2022       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 470.57.02    Driver Version: 470.57.02    CUDA Version: 11.4     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla V100-SXM2...  On   | 00000000:00:05.0 Off |                  Off |
| N/A   43C    P0    66W / 300W |  32506MiB / 32510MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
|   1  Tesla V100-SXM2...  On   | 00000000:00:06.0 Off |                  Off |
| N/A   

In [4]:
gpus = tf.config.experimental.list_physical_devices('GPU')
if gpus:
    try:
        tf.config.experimental.set_visible_devices(gpus[1], 'GPU')
    except RuntimeError as e:
        print(e)

In [5]:
data_folder =  'physionet.org/files/circor-heart-sound/1.0.3/training_data'
train_folder =  '/home/ubuntu/data/hmd/murmur/train'
test_folder = '/home/ubuntu/data/hmd/murmur/test'


In [6]:
model_folder = 'lcnn2'

## Modeling



In [7]:
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.callbacks import LearningRateScheduler, ModelCheckpoint

In [8]:
def sigmoidal_decay(e, start=0, end=100, lr_start=1e-3, lr_end=1e-5):
    if e < start:
        return lr_start
    elif e > end:
        return lr_end

    middle = (start + end) / 2
    s = lambda x: 1 / (1 + np.exp(-x))

    return s(13 * (-e + middle) / np.abs(end - start)) * np.abs(lr_start - lr_end) + lr_end

### get feature 함수확장: 음성피쳐 옵션들과, 추가 음성들 고려한 피쳐추출

In [9]:
patient_files_trn = find_patient_files(train_folder)
patient_files_test = find_patient_files(test_folder)

In [10]:
params_feature = {'samp_sec': 20,
                  #### melspec, stft 피쳐 옵션들  
                  'pre_emphasis': 0,
                  'hop_length': 256,
                  'win_length':512,
                  'n_mels': 100,
                  #### cqt 피쳐 옵션들  
                  'filter_scale': 1,
                  'n_bins': 80,
                  'fmin': 10
}

In [11]:
features_trn, mm_lbs_trn, out_lbs_trn, mel_input_shape, cqt_input_shape, stft_input_shape = get_features_3lb_all(train_folder, patient_files_trn, **params_feature)
features_test, mm_lbs_test, out_lbs_test, _, _, _ = get_features_3lb_all(test_folder, patient_files_test, **params_feature)

melspec:  100 313
cqt:  80 157
stft:  257 313


utils/get_feature.py:599: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  features[k1] = np.array(features[k1])


melspec:  100 313
cqt:  80 157
stft:  257 313


### LCNN


LCNN 초기모형 작성

In [75]:

from tensorflow.keras.models import Model
from tensorflow.keras.layers import Dense, Dropout, Activation, Flatten
from tensorflow.keras.layers import Input, Conv2D, MaxPooling2D, BatchNormalization
from tensorflow.keras.layers import Convolution2D, GlobalAveragePooling2D, MaxPool2D

def get_LCNN_o_1(mel_input_shape, cqt_input_shape, stft_input_shape, use_mel = True, use_cqt = True, use_stft = True, ord1 = True):
        # Create a towy model.
    age = keras.Input(shape=(6,), name = 'age_cat')
    sex = keras.Input(shape=(2,), name = 'sex_cat')
    hw = keras.Input(shape=(2,), name = 'height_weight')
    preg = keras.Input(shape=(1,), name = 'is_preg')
    loc = keras.Input(shape=(5,), name = 'loc')
    mel1 = keras.Input(shape=(mel_input_shape), name = 'mel')
    cqt1 = keras.Input(shape=(cqt_input_shape), name = 'cqt')
    stft1 = keras.Input(shape=(stft_input_shape), name = 'stft')
        
        
    ## age embeddig
    age1 = layers.Dense(2, activation = None)(age)

    ## sex embedding
    sex1 = layers.Dense(1, activation = None)(sex)

    ## hw embedding
    hw1 = layers.Dense(1, activation = None)(hw)

    ## loc embedding
    loc1 = layers.Dense(3, activation = None)(loc)

    ## mel embedding
    conv1_1 = Conv2D(filters = 32, kernel_size =5, strides=(1, 1), padding='same', activation=None)(mel1)
    conv1_2 = Conv2D(filters = 32, kernel_size =5, strides=(1, 1), padding='same', activation=None)(mel1)
    mfm2 = tensorflow.keras.layers.maximum([conv1_1, conv1_2])
    max3 = MaxPool2D(pool_size=(2, 2), strides=(2, 2), padding='same')(mfm2)

    conv4_1 = Conv2D(filters = 32, kernel_size =1, strides=(1, 1), padding='same', activation=None)(max3)
    conv4_2 = Conv2D(filters = 32, kernel_size =1, strides=(1, 1), padding='same', activation=None)(max3)
    mfm5 = tensorflow.keras.layers.maximum([conv4_1, conv4_2])
    batch6 = BatchNormalization(axis=3, scale=False)(mfm5)

    conv7_1 = Conv2D(filters = 48, kernel_size =3, strides=(1, 1), padding='same', activation=None)(batch6)
    conv7_2 = Conv2D(filters = 48, kernel_size =3, strides=(1, 1), padding='same', activation=None)(batch6)
    mfm8 = tensorflow.keras.layers.maximum([conv7_1, conv7_2])

    max9 = MaxPool2D(pool_size=(2, 2), strides=(2, 2), padding='same')(mfm8)
    batch10 = BatchNormalization(axis=3, scale=False)(max9)

    conv11_1 = Conv2D(filters = 48, kernel_size =1, strides=(1, 1), padding='same', activation=None)(batch10)
    conv11_2 = Conv2D(filters = 48, kernel_size =1, strides=(1, 1), padding='same', activation=None)(batch10)
    mfm12 = tensorflow.keras.layers.maximum([conv11_1, conv11_2])
    batch13 = BatchNormalization(axis=3, scale=False)(mfm12)

    conv14_1 = Conv2D(filters = 64, kernel_size =3, strides=(1, 1), padding='same', activation=None)(batch13)
    conv14_2 = Conv2D(filters = 64, kernel_size =3, strides=(1, 1), padding='same', activation=None)(batch13)
    mfm15 = tensorflow.keras.layers.maximum([conv14_1, conv14_2])

    max16 = MaxPool2D(pool_size=(2, 2), strides=(2, 2), padding='same')(mfm15)

    conv17_1 = Conv2D(filters = 64, kernel_size =1, strides=(1, 1), padding='same', activation=None)(max16)
    conv17_2 = Conv2D(filters = 64, kernel_size =1, strides=(1, 1), padding='same', activation=None)(max16)
    mfm18 = tensorflow.keras.layers.maximum([conv17_1, conv17_2])
    batch19 = BatchNormalization(axis=3, scale=False)(mfm18)

    conv20_1 = Conv2D(filters = 32, kernel_size =3, strides=(1, 1), padding='same', activation=None)(batch19)
    conv20_2 = Conv2D(filters = 32, kernel_size =3, strides=(1, 1), padding='same', activation=None)(batch19)
    mfm21 = tensorflow.keras.layers.maximum([conv20_1, conv20_2])
    batch22 = BatchNormalization(axis=3, scale=False)(mfm21)

    conv23_1 = Conv2D(filters = 32, kernel_size =1, strides=(1, 1), padding='same', activation=None)(batch22)
    conv23_2 = Conv2D(filters = 32, kernel_size =1, strides=(1, 1), padding='same', activation=None)(batch22)
    mfm24 = tensorflow.keras.layers.maximum([conv23_1, conv23_2])
    batch25 = BatchNormalization(axis=3, scale=False)(mfm24)

    conv26_1 = Conv2D(filters = 32, kernel_size =1, strides=(1, 1), padding='same', activation=None)(batch25)
    conv26_2 = Conv2D(filters = 32, kernel_size =1, strides=(1, 1), padding='same', activation=None)(batch25)
    mfm27 = tensorflow.keras.layers.maximum([conv26_1, conv26_2])

    max28 = MaxPool2D(pool_size=(2, 2), strides=(2, 2), padding='valid')(mfm27)
    mel2 = layers.GlobalAveragePooling2D()(max28)

    ## cqt embedding
    conv1_1 = Conv2D(filters = 32, kernel_size =5, strides=(1, 1), padding='same', activation=None)(cqt1)
    conv1_2 = Conv2D(filters = 32, kernel_size =5, strides=(1, 1), padding='same', activation=None)(cqt1)
    mfm2 = tensorflow.keras.layers.maximum([conv1_1, conv1_2])
    max3 = MaxPool2D(pool_size=(2, 2), strides=(2, 2), padding='same')(mfm2)

    conv4_1 = Conv2D(filters = 32, kernel_size =1, strides=(1, 1), padding='same', activation=None)(max3)
    conv4_2 = Conv2D(filters = 32, kernel_size =1, strides=(1, 1), padding='same', activation=None)(max3)
    mfm5 = tensorflow.keras.layers.maximum([conv4_1, conv4_2])
    batch6 = BatchNormalization(axis=3, scale=False)(mfm5)

    conv7_1 = Conv2D(filters = 48, kernel_size =3, strides=(1, 1), padding='same', activation=None)(batch6)
    conv7_2 = Conv2D(filters = 48, kernel_size =3, strides=(1, 1), padding='same', activation=None)(batch6)
    mfm8 = tensorflow.keras.layers.maximum([conv7_1, conv7_2])

    max9 = MaxPool2D(pool_size=(2, 2), strides=(2, 2), padding='same')(mfm8)
    batch10 = BatchNormalization(axis=3, scale=False)(max9)

    conv11_1 = Conv2D(filters = 48, kernel_size =1, strides=(1, 1), padding='same', activation=None)(batch10)
    conv11_2 = Conv2D(filters = 48, kernel_size =1, strides=(1, 1), padding='same', activation=None)(batch10)
    mfm12 = tensorflow.keras.layers.maximum([conv11_1, conv11_2])
    batch13 = BatchNormalization(axis=3, scale=False)(mfm12)

    conv14_1 = Conv2D(filters = 64, kernel_size =3, strides=(1, 1), padding='same', activation=None)(batch13)
    conv14_2 = Conv2D(filters = 64, kernel_size =3, strides=(1, 1), padding='same', activation=None)(batch13)
    mfm15 = tensorflow.keras.layers.maximum([conv14_1, conv14_2])

    max16 = MaxPool2D(pool_size=(2, 2), strides=(2, 2), padding='same')(mfm15)

    conv17_1 = Conv2D(filters = 64, kernel_size =1, strides=(1, 1), padding='same', activation=None)(max16)
    conv17_2 = Conv2D(filters = 64, kernel_size =1, strides=(1, 1), padding='same', activation=None)(max16)
    mfm18 = tensorflow.keras.layers.maximum([conv17_1, conv17_2])
    batch19 = BatchNormalization(axis=3, scale=False)(mfm18)

    conv20_1 = Conv2D(filters = 32, kernel_size =3, strides=(1, 1), padding='same', activation=None)(batch19)
    conv20_2 = Conv2D(filters = 32, kernel_size =3, strides=(1, 1), padding='same', activation=None)(batch19)
    mfm21 = tensorflow.keras.layers.maximum([conv20_1, conv20_2])
    batch22 = BatchNormalization(axis=3, scale=False)(mfm21)

    conv23_1 = Conv2D(filters = 32, kernel_size =3, strides=(1, 1), padding='same', activation=None)(batch22)
    conv23_2 = Conv2D(filters = 32, kernel_size =3, strides=(1, 1), padding='same', activation=None)(batch22)
    mfm24 = tensorflow.keras.layers.maximum([conv23_1, conv23_2])
    batch25 = BatchNormalization(axis=3, scale=False)(mfm24)

    conv26_1 = Conv2D(filters = 32, kernel_size =3, strides=(1, 1), padding='same', activation=None)(batch25)
    conv26_2 = Conv2D(filters = 32, kernel_size =3, strides=(1, 1), padding='same', activation=None)(batch25)
    mfm27 = tensorflow.keras.layers.maximum([conv26_1, conv26_2])

    max28 = MaxPool2D(pool_size=(2, 2), strides=(2, 2), padding='valid')(mfm27)
    cqt2 = layers.GlobalAveragePooling2D()(max28)

    ## stft embedding
    conv1_1 = Conv2D(filters = 32, kernel_size =5, strides=(1, 1), padding='same', activation=None)(stft1)
    conv1_2 = Conv2D(filters = 32, kernel_size =5, strides=(1, 1), padding='same', activation=None)(stft1)
    mfm2 = tensorflow.keras.layers.maximum([conv1_1, conv1_2])
    max3 = MaxPool2D(pool_size=(2, 2), strides=(2, 2), padding='same')(mfm2)

    conv4_1 = Conv2D(filters = 32, kernel_size =1, strides=(1, 1), padding='same', activation=None)(max3)
    conv4_2 = Conv2D(filters = 32, kernel_size =1, strides=(1, 1), padding='same', activation=None)(max3)
    mfm5 = tensorflow.keras.layers.maximum([conv4_1, conv4_2])
    batch6 = BatchNormalization(axis=3, scale=False)(mfm5)

    conv7_1 = Conv2D(filters = 48, kernel_size =3, strides=(1, 1), padding='same', activation=None)(batch6)
    conv7_2 = Conv2D(filters = 48, kernel_size =3, strides=(1, 1), padding='same', activation=None)(batch6)
    mfm8 = tensorflow.keras.layers.maximum([conv7_1, conv7_2])

    max9 = MaxPool2D(pool_size=(2, 2), strides=(2, 2), padding='same')(mfm8)
    batch10 = BatchNormalization(axis=3, scale=False)(max9)

    conv11_1 = Conv2D(filters = 48, kernel_size =1, strides=(1, 1), padding='same', activation=None)(batch10)
    conv11_2 = Conv2D(filters = 48, kernel_size =1, strides=(1, 1), padding='same', activation=None)(batch10)
    mfm12 = tensorflow.keras.layers.maximum([conv11_1, conv11_2])
    batch13 = BatchNormalization(axis=3, scale=False)(mfm12)

    conv14_1 = Conv2D(filters = 64, kernel_size =3, strides=(1, 1), padding='same', activation=None)(batch13)
    conv14_2 = Conv2D(filters = 64, kernel_size =3, strides=(1, 1), padding='same', activation=None)(batch13)
    mfm15 = tensorflow.keras.layers.maximum([conv14_1, conv14_2])

    max16 = MaxPool2D(pool_size=(2, 2), strides=(2, 2), padding='same')(mfm15)

    conv17_1 = Conv2D(filters = 64, kernel_size =1, strides=(1, 1), padding='same', activation=None)(max16)
    conv17_2 = Conv2D(filters = 64, kernel_size =1, strides=(1, 1), padding='same', activation=None)(max16)
    mfm18 = tensorflow.keras.layers.maximum([conv17_1, conv17_2])
    batch19 = BatchNormalization(axis=3, scale=False)(mfm18)

    conv20_1 = Conv2D(filters = 32, kernel_size =3, strides=(1, 1), padding='same', activation=None)(batch19)
    conv20_2 = Conv2D(filters = 32, kernel_size =3, strides=(1, 1), padding='same', activation=None)(batch19)
    mfm21 = tensorflow.keras.layers.maximum([conv20_1, conv20_2])
    batch22 = BatchNormalization(axis=3, scale=False)(mfm21)

    conv23_1 = Conv2D(filters = 32, kernel_size =3, strides=(1, 1), padding='same', activation=None)(batch22)
    conv23_2 = Conv2D(filters = 32, kernel_size =3, strides=(1, 1), padding='same', activation=None)(batch22)
    mfm24 = tensorflow.keras.layers.maximum([conv23_1, conv23_2])
    batch25 = BatchNormalization(axis=3, scale=False)(mfm24)

    conv26_1 = Conv2D(filters = 32, kernel_size =3, strides=(1, 1), padding='same', activation=None)(batch25)
    conv26_2 = Conv2D(filters = 32, kernel_size =3, strides=(1, 1), padding='same', activation=None)(batch25)
    mfm27 = tensorflow.keras.layers.maximum([conv26_1, conv26_2])

    max28 = MaxPool2D(pool_size=(2, 2), strides=(2, 2), padding='valid')(mfm27)
    stft2 = layers.GlobalAveragePooling2D()(max28)
    
#    concat1 = layers.Concatenate()([age1, sex1, hw1, loc1, preg])
#    d1 = layers.Dense(2, activation = 'relu')(concat1)
    
    if use_mel and use_cqt and use_stft :
        concat2 = layers.Concatenate()([mel2, cqt2, stft2])
    if not use_mel and use_cqt and use_stft :
        concat2 = layers.Concatenate()([cqt2, stft2])
    if use_mel and not use_cqt and use_stft :
        concat2 = layers.Concatenate()([mel2, stft2])
    if use_mel and use_cqt and not use_stft :
        concat2 = layers.Concatenate()([mel2, cqt2])
    if not use_mel and not use_cqt and use_stft :  ## stft 만
        concat2 = stft2
    if use_mel and not use_cqt and not use_stft :  ### mel만
        concat2 = mel2
    if not use_mel and use_cqt and not use_stft :  ### cqt만
        concat2 = cqt2
    if ord1 :
        res1 = layers.Dense(2, activation = "softmax")(concat2)
    else :
        res1 = layers.Dense(3, activation = "softmax")(concat2)
        
    res2 = layers.Dense(2, activation = "softmax")(concat2)

    model = keras.Model(inputs = [age,sex,hw,preg,loc,mel1,cqt1,stft1] , outputs = res1 )
    
    model.compile(loss=['categorical_crossentropy'], optimizer='adam', metrics=['accuracy','AUC'])
    return(model)

def get_LCNN_2(mel_input_shape, cqt_input_shape, stft_input_shape, use_mel = True, use_cqt = True, use_stft = True):
        # Create a towy model.
    age = keras.Input(shape=(6,), name = 'age_cat')
    sex = keras.Input(shape=(2,), name = 'sex_cat')
    hw = keras.Input(shape=(2,), name = 'height_weight')
    preg = keras.Input(shape=(1,), name = 'is_preg')
    loc = keras.Input(shape=(5,), name = 'loc')
    mel1 = keras.Input(shape=(mel_input_shape), name = 'mel')
    cqt1 = keras.Input(shape=(cqt_input_shape), name = 'cqt')
    stft1 = keras.Input(shape=(stft_input_shape), name = 'stft')
        
        
    ## age embeddig
    age1 = layers.Dense(2, activation = None)(age)

    ## sex embedding
    sex1 = layers.Dense(1, activation = None)(sex)

    ## hw embedding
    hw1 = layers.Dense(1, activation = None)(hw)

    ## loc embedding
    loc1 = layers.Dense(3, activation = None)(loc)

    ## mel embedding
    conv1_1 = Conv2D(filters = 32, kernel_size =5, strides=(1, 1), padding='same', activation=None)(mel1)
    conv1_2 = Conv2D(filters = 32, kernel_size =5, strides=(1, 1), padding='same', activation=None)(mel1)
    mfm2 = tensorflow.keras.layers.maximum([conv1_1, conv1_2])
    max3 = MaxPool2D(pool_size=(2, 2), strides=(2, 2), padding='same')(mfm2)

    conv4_1 = Conv2D(filters = 32, kernel_size =1, strides=(1, 1), padding='same', activation=None)(max3)
    conv4_2 = Conv2D(filters = 32, kernel_size =1, strides=(1, 1), padding='same', activation=None)(max3)
    mfm5 = tensorflow.keras.layers.maximum([conv4_1, conv4_2])
    batch6 = BatchNormalization(axis=3, scale=False)(mfm5)

    conv7_1 = Conv2D(filters = 48, kernel_size =3, strides=(1, 1), padding='same', activation=None)(batch6)
    conv7_2 = Conv2D(filters = 48, kernel_size =3, strides=(1, 1), padding='same', activation=None)(batch6)
    mfm8 = tensorflow.keras.layers.maximum([conv7_1, conv7_2])

    max9 = MaxPool2D(pool_size=(2, 2), strides=(2, 2), padding='same')(mfm8)
    batch10 = BatchNormalization(axis=3, scale=False)(max9)

    conv11_1 = Conv2D(filters = 48, kernel_size =1, strides=(1, 1), padding='same', activation=None)(batch10)
    conv11_2 = Conv2D(filters = 48, kernel_size =1, strides=(1, 1), padding='same', activation=None)(batch10)
    mfm12 = tensorflow.keras.layers.maximum([conv11_1, conv11_2])
    batch13 = BatchNormalization(axis=3, scale=False)(mfm12)

    conv14_1 = Conv2D(filters = 64, kernel_size =3, strides=(1, 1), padding='same', activation=None)(batch13)
    conv14_2 = Conv2D(filters = 64, kernel_size =3, strides=(1, 1), padding='same', activation=None)(batch13)
    mfm15 = tensorflow.keras.layers.maximum([conv14_1, conv14_2])

    max16 = MaxPool2D(pool_size=(2, 2), strides=(2, 2), padding='same')(mfm15)

    conv17_1 = Conv2D(filters = 64, kernel_size =1, strides=(1, 1), padding='same', activation=None)(max16)
    conv17_2 = Conv2D(filters = 64, kernel_size =1, strides=(1, 1), padding='same', activation=None)(max16)
    mfm18 = tensorflow.keras.layers.maximum([conv17_1, conv17_2])
    batch19 = BatchNormalization(axis=3, scale=False)(mfm18)

    conv20_1 = Conv2D(filters = 32, kernel_size =3, strides=(1, 1), padding='same', activation=None)(batch19)
    conv20_2 = Conv2D(filters = 32, kernel_size =3, strides=(1, 1), padding='same', activation=None)(batch19)
    mfm21 = tensorflow.keras.layers.maximum([conv20_1, conv20_2])
    batch22 = BatchNormalization(axis=3, scale=False)(mfm21)

    conv23_1 = Conv2D(filters = 32, kernel_size =3, strides=(1, 1), padding='same', activation=None)(batch22)
    conv23_2 = Conv2D(filters = 32, kernel_size =3, strides=(1, 1), padding='same', activation=None)(batch22)
    mfm24 = tensorflow.keras.layers.maximum([conv23_1, conv23_2])
    batch25 = BatchNormalization(axis=3, scale=False)(mfm24)

    conv26_1 = Conv2D(filters = 32, kernel_size =3, strides=(1, 1), padding='same', activation=None)(batch25)
    conv26_2 = Conv2D(filters = 32, kernel_size =3, strides=(1, 1), padding='same', activation=None)(batch25)
    mfm27 = tensorflow.keras.layers.maximum([conv26_1, conv26_2])

    max28 = MaxPool2D(pool_size=(2, 2), strides=(2, 2), padding='valid')(mfm27)
    mel2 = layers.GlobalAveragePooling2D()(max28)

    ## cqt embedding
    conv1_1 = Conv2D(filters = 32, kernel_size =5, strides=(1, 1), padding='same', activation=None)(cqt1)
    conv1_2 = Conv2D(filters = 32, kernel_size =5, strides=(1, 1), padding='same', activation=None)(cqt1)
    mfm2 = tensorflow.keras.layers.maximum([conv1_1, conv1_2])
    max3 = MaxPool2D(pool_size=(2, 2), strides=(2, 2), padding='same')(mfm2)

    conv4_1 = Conv2D(filters = 32, kernel_size =1, strides=(1, 1), padding='same', activation=None)(max3)
    conv4_2 = Conv2D(filters = 32, kernel_size =1, strides=(1, 1), padding='same', activation=None)(max3)
    mfm5 = tensorflow.keras.layers.maximum([conv4_1, conv4_2])
    batch6 = BatchNormalization(axis=3, scale=False)(mfm5)

    conv7_1 = Conv2D(filters = 48, kernel_size =3, strides=(1, 1), padding='same', activation=None)(batch6)
    conv7_2 = Conv2D(filters = 48, kernel_size =3, strides=(1, 1), padding='same', activation=None)(batch6)
    mfm8 = tensorflow.keras.layers.maximum([conv7_1, conv7_2])

    max9 = MaxPool2D(pool_size=(2, 2), strides=(2, 2), padding='same')(mfm8)
    batch10 = BatchNormalization(axis=3, scale=False)(max9)

    conv11_1 = Conv2D(filters = 48, kernel_size =1, strides=(1, 1), padding='same', activation=None)(batch10)
    conv11_2 = Conv2D(filters = 48, kernel_size =1, strides=(1, 1), padding='same', activation=None)(batch10)
    mfm12 = tensorflow.keras.layers.maximum([conv11_1, conv11_2])
    batch13 = BatchNormalization(axis=3, scale=False)(mfm12)

    conv14_1 = Conv2D(filters = 64, kernel_size =3, strides=(1, 1), padding='same', activation=None)(batch13)
    conv14_2 = Conv2D(filters = 64, kernel_size =3, strides=(1, 1), padding='same', activation=None)(batch13)
    mfm15 = tensorflow.keras.layers.maximum([conv14_1, conv14_2])

    max16 = MaxPool2D(pool_size=(2, 2), strides=(2, 2), padding='same')(mfm15)

    conv17_1 = Conv2D(filters = 64, kernel_size =1, strides=(1, 1), padding='same', activation=None)(max16)
    conv17_2 = Conv2D(filters = 64, kernel_size =1, strides=(1, 1), padding='same', activation=None)(max16)
    mfm18 = tensorflow.keras.layers.maximum([conv17_1, conv17_2])
    batch19 = BatchNormalization(axis=3, scale=False)(mfm18)

    conv20_1 = Conv2D(filters = 32, kernel_size =3, strides=(1, 1), padding='same', activation=None)(batch19)
    conv20_2 = Conv2D(filters = 32, kernel_size =3, strides=(1, 1), padding='same', activation=None)(batch19)
    mfm21 = tensorflow.keras.layers.maximum([conv20_1, conv20_2])
    batch22 = BatchNormalization(axis=3, scale=False)(mfm21)

    conv23_1 = Conv2D(filters = 32, kernel_size =3, strides=(1, 1), padding='same', activation=None)(batch22)
    conv23_2 = Conv2D(filters = 32, kernel_size =3, strides=(1, 1), padding='same', activation=None)(batch22)
    mfm24 = tensorflow.keras.layers.maximum([conv23_1, conv23_2])
    batch25 = BatchNormalization(axis=3, scale=False)(mfm24)

    conv26_1 = Conv2D(filters = 32, kernel_size =3, strides=(1, 1), padding='same', activation=None)(batch25)
    conv26_2 = Conv2D(filters = 32, kernel_size =3, strides=(1, 1), padding='same', activation=None)(batch25)
    mfm27 = tensorflow.keras.layers.maximum([conv26_1, conv26_2])

    max28 = MaxPool2D(pool_size=(2, 2), strides=(2, 2), padding='valid')(mfm27)
    cqt2 = layers.GlobalAveragePooling2D()(max28)

    ## stft embedding
    conv1_1 = Conv2D(filters = 32, kernel_size =5, strides=(1, 1), padding='same', activation=None)(stft1)
    conv1_2 = Conv2D(filters = 32, kernel_size =5, strides=(1, 1), padding='same', activation=None)(stft1)
    mfm2 = tensorflow.keras.layers.maximum([conv1_1, conv1_2])
    max3 = MaxPool2D(pool_size=(2, 2), strides=(2, 2), padding='same')(mfm2)

    conv4_1 = Conv2D(filters = 32, kernel_size =1, strides=(1, 1), padding='same', activation=None)(max3)
    conv4_2 = Conv2D(filters = 32, kernel_size =1, strides=(1, 1), padding='same', activation=None)(max3)
    mfm5 = tensorflow.keras.layers.maximum([conv4_1, conv4_2])
    batch6 = BatchNormalization(axis=3, scale=False)(mfm5)

    conv7_1 = Conv2D(filters = 48, kernel_size =3, strides=(1, 1), padding='same', activation=None)(batch6)
    conv7_2 = Conv2D(filters = 48, kernel_size =3, strides=(1, 1), padding='same', activation=None)(batch6)
    mfm8 = tensorflow.keras.layers.maximum([conv7_1, conv7_2])

    max9 = MaxPool2D(pool_size=(2, 2), strides=(2, 2), padding='same')(mfm8)
    batch10 = BatchNormalization(axis=3, scale=False)(max9)

    conv11_1 = Conv2D(filters = 48, kernel_size =1, strides=(1, 1), padding='same', activation=None)(batch10)
    conv11_2 = Conv2D(filters = 48, kernel_size =1, strides=(1, 1), padding='same', activation=None)(batch10)
    mfm12 = tensorflow.keras.layers.maximum([conv11_1, conv11_2])
    batch13 = BatchNormalization(axis=3, scale=False)(mfm12)

    conv14_1 = Conv2D(filters = 64, kernel_size =3, strides=(1, 1), padding='same', activation=None)(batch13)
    conv14_2 = Conv2D(filters = 64, kernel_size =3, strides=(1, 1), padding='same', activation=None)(batch13)
    mfm15 = tensorflow.keras.layers.maximum([conv14_1, conv14_2])

    max16 = MaxPool2D(pool_size=(2, 2), strides=(2, 2), padding='same')(mfm15)

    conv17_1 = Conv2D(filters = 64, kernel_size =1, strides=(1, 1), padding='same', activation=None)(max16)
    conv17_2 = Conv2D(filters = 64, kernel_size =1, strides=(1, 1), padding='same', activation=None)(max16)
    mfm18 = tensorflow.keras.layers.maximum([conv17_1, conv17_2])
    batch19 = BatchNormalization(axis=3, scale=False)(mfm18)

    conv20_1 = Conv2D(filters = 32, kernel_size =3, strides=(1, 1), padding='same', activation=None)(batch19)
    conv20_2 = Conv2D(filters = 32, kernel_size =3, strides=(1, 1), padding='same', activation=None)(batch19)
    mfm21 = tensorflow.keras.layers.maximum([conv20_1, conv20_2])
    batch22 = BatchNormalization(axis=3, scale=False)(mfm21)

    conv23_1 = Conv2D(filters = 32, kernel_size =3, strides=(1, 1), padding='same', activation=None)(batch22)
    conv23_2 = Conv2D(filters = 32, kernel_size =3, strides=(1, 1), padding='same', activation=None)(batch22)
    mfm24 = tensorflow.keras.layers.maximum([conv23_1, conv23_2])
    batch25 = BatchNormalization(axis=3, scale=False)(mfm24)

    conv26_1 = Conv2D(filters = 32, kernel_size =3, strides=(1, 1), padding='same', activation=None)(batch25)
    conv26_2 = Conv2D(filters = 32, kernel_size =3, strides=(1, 1), padding='same', activation=None)(batch25)
    mfm27 = tensorflow.keras.layers.maximum([conv26_1, conv26_2])

    max28 = MaxPool2D(pool_size=(2, 2), strides=(2, 2), padding='valid')(mfm27)
    stft2 = layers.GlobalAveragePooling2D()(max28)
    
#    concat1 = layers.Concatenate()([age1, sex1, hw1, loc1, preg])
#    d1 = layers.Dense(2, activation = 'relu')(concat1)
    
    if use_mel and use_cqt and use_stft :
        concat2 = layers.Concatenate()([mel2, cqt2, stft2])
    if not use_mel and use_cqt and use_stft :
        concat2 = layers.Concatenate()([cqt2, stft2])
    if use_mel and not use_cqt and use_stft :
        concat2 = layers.Concatenate()([mel2, stft2])
    if use_mel and use_cqt and not use_stft :
        concat2 = layers.Concatenate()([mel2, cqt2])
    if not use_mel and not use_cqt and use_stft :  ## stft 만
        concat2 = stft2
    if use_mel and not use_cqt and not use_stft :  ### mel만
        concat2 = mel2
    if not use_mel and use_cqt and not use_stft :  ### cqt만
        concat2 = cqt2
#    concat2 = layers.Dense(10, activation = 'relu')(concat2)
    res1 = layers.Dense(3, activation = "softmax")(concat2)
    res2 = layers.Dense(2, activation = "softmax")(concat2)

    model = keras.Model(inputs = [age,sex,hw,preg,loc,mel1,cqt1,stft1] , outputs = res2 )
    
    model.compile(loss=['categorical_crossentropy'], optimizer='adam', metrics=['accuracy','AUC'])
    return(model)

In [76]:
#model1 = get_toy5_1(mel_input_shape, cqt_input_shape, stft_input_shape)
#model2 = get_toy5_2(mel_input_shape, cqt_input_shape, stft_input_shape)


In [77]:
model1 = get_LCNN_1(mel_input_shape, cqt_input_shape, stft_input_shape, use_mel = True, use_cqt = False, use_stft = False)
model2 = get_LCNN_2(mel_input_shape, cqt_input_shape, stft_input_shape, use_mel = True, use_cqt = False, use_stft = False)


In [78]:
model1.summary()

Model: "model_1150"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
mel (InputLayer)                [(None, 100, 313, 1) 0                                            
__________________________________________________________________________________________________
conv2d_62100 (Conv2D)           (None, 100, 313, 32) 832         mel[0][0]                        
__________________________________________________________________________________________________
conv2d_62101 (Conv2D)           (None, 100, 313, 32) 832         mel[0][0]                        
__________________________________________________________________________________________________
maximum_31050 (Maximum)         (None, 100, 313, 32) 0           conv2d_62100[0][0]               
                                                                 conv2d_62101[0][0]      

In [79]:
n_epoch = 100
lr = LearningRateScheduler(lambda e: sigmoidal_decay(e, end=n_epoch))
batch_size = 64
params = {'batch_size': batch_size,
#          'input_shape': (100, 313, 1),
          'shuffle': True,
          'beta_param': 0.7,
          'mixup': True,
#          'lowpass': [.5, [11,12,13,14,15,16,17,18]]
#          'highpass': [.5, [78,79,80,81,82,83,84,85]]
          'ranfilter2' : [3, [18,19,20,21,22,23]]
#           'dropblock' : [30, 100]
          #'device' : device
}

params_no_shuffle = {'batch_size': batch_size,
#          'input_shape': (100, 313, 1),
          'shuffle': False,
          'beta_param': 0.7,
          'mixup': False
          #'device': device
}

TrainDGen_1 = Generator0([features_trn['age'],features_trn['sex'], features_trn['hw'], features_trn['preg'], features_trn['loc'], 
           features_trn['mel1'],features_trn['cqt1'],features_trn['stft1']], 
                        mm_lbs_trn,  ## our Y
                        **params)()

#ValDGen_1 = Generator0([features_test[0]['age'],features_test[0]['sex'], features_test[0]['hw'], features_test[0]['preg'], features_test[0]['loc'], 
#           features_test[0]['mel1'],features_test[0]['cqt1'],features_test[0]['stft1']], 
#                        mm_lbs_test,  ## our Y
#                        **params_no_shuffle)()

class_weight = {0: 3, 1: 1., 2: 1} 
    
model1.fit(TrainDGen_1,
          validation_data = ([features_test['age'],features_test['sex'], features_test['hw'], 
                              features_test['preg'], features_test['loc'], features_test['mel1'], 
                              features_test['cqt1'], features_test['stft1']], 
                             mm_lbs_test), 
                             callbacks=[lr],
                              steps_per_epoch=np.ceil(len(mm_lbs_trn)/64),
                           class_weight=class_weight, 
                             epochs = n_epoch)

Epoch 1/100
40/40 [==============================] - 9s 192ms/step - loss: 1.2424 - accuracy: 0.6624 - auc: 0.8125 - val_loss: 9.8755 - val_accuracy: 0.2552 - val_auc: 0.4040
Epoch 2/100
40/40 [==============================] - 7s 169ms/step - loss: 1.0923 - accuracy: 0.6913 - auc: 0.8460 - val_loss: 10.0003 - val_accuracy: 0.2552 - val_auc: 0.4049
Epoch 3/100
40/40 [==============================] - 7s 168ms/step - loss: 1.1246 - accuracy: 0.6913 - auc: 0.8463 - val_loss: 2.3272 - val_accuracy: 0.2583 - val_auc: 0.5237
Epoch 4/100
40/40 [==============================] - 7s 167ms/step - loss: 1.0766 - accuracy: 0.7205 - auc: 0.8621 - val_loss: 1.2448 - val_accuracy: 0.2995 - val_auc: 0.5532
Epoch 5/100
40/40 [==============================] - 7s 167ms/step - loss: 1.0267 - accuracy: 0.7589 - auc: 0.8723 - val_loss: 0.7467 - val_accuracy: 0.5721 - val_auc: 0.7566
Epoch 6/100
40/40 [==============================] - 7s 171ms/step - loss: 1.0630 - accuracy: 0.7332 - auc: 0.8642 - val_los

40/40 [==============================] - 6s 165ms/step - loss: 1.0337 - accuracy: 0.7603 - auc: 0.8742 - val_loss: 0.5036 - val_accuracy: 0.8479 - val_auc: 0.9210
Epoch 48/100
40/40 [==============================] - 7s 167ms/step - loss: 1.0291 - accuracy: 0.7649 - auc: 0.8798 - val_loss: 0.5390 - val_accuracy: 0.8431 - val_auc: 0.9160
Epoch 49/100
40/40 [==============================] - 7s 168ms/step - loss: 0.9594 - accuracy: 0.7794 - auc: 0.8851 - val_loss: 0.8589 - val_accuracy: 0.3772 - val_auc: 0.6616
Epoch 50/100
40/40 [==============================] - 7s 166ms/step - loss: 0.9767 - accuracy: 0.7637 - auc: 0.8903 - val_loss: 0.5528 - val_accuracy: 0.8288 - val_auc: 0.9097
Epoch 51/100
40/40 [==============================] - 7s 167ms/step - loss: 1.0048 - accuracy: 0.7649 - auc: 0.8758 - val_loss: 0.4853 - val_accuracy: 0.8479 - val_auc: 0.9214
Epoch 52/100
40/40 [==============================] - 7s 170ms/step - loss: 1.0170 - accuracy: 0.7623 - auc: 0.8843 - val_loss: 0.503

Epoch 94/100
40/40 [==============================] - 6s 163ms/step - loss: 0.9583 - accuracy: 0.7877 - auc: 0.8970 - val_loss: 0.5009 - val_accuracy: 0.8463 - val_auc: 0.9151
Epoch 95/100
40/40 [==============================] - 6s 163ms/step - loss: 0.9891 - accuracy: 0.7859 - auc: 0.8946 - val_loss: 0.5041 - val_accuracy: 0.8494 - val_auc: 0.9168
Epoch 96/100
40/40 [==============================] - 7s 173ms/step - loss: 1.0170 - accuracy: 0.7707 - auc: 0.8923 - val_loss: 0.5079 - val_accuracy: 0.8494 - val_auc: 0.9162
Epoch 97/100
40/40 [==============================] - 7s 168ms/step - loss: 0.9123 - accuracy: 0.8142 - auc: 0.9029 - val_loss: 0.4925 - val_accuracy: 0.8574 - val_auc: 0.9186
Epoch 98/100
40/40 [==============================] - 6s 158ms/step - loss: 0.9909 - accuracy: 0.7781 - auc: 0.8884 - val_loss: 0.5064 - val_accuracy: 0.8447 - val_auc: 0.9158
Epoch 99/100
40/40 [==============================] - 7s 169ms/step - loss: 0.9830 - accuracy: 0.7632 - auc: 0.8964 - va

트레이닝 accuracy 와 auc 가 validation accuracy, auc 와 거의 유사함

In [80]:
n_epoch = 100
lr = LearningRateScheduler(lambda e: sigmoidal_decay(e, end=n_epoch))
batch_size = 64
params = {'batch_size': batch_size,
#          'input_shape': (100, 313, 1),
          'shuffle': True,
          'beta_param': 0.7,
          'mixup': True,
#          'lowpass': [.5, [11,12,13,14,15,16,17,18]]
          'highpass': [.5, [78,79,80,81,82,83,84,85]],
          'ranfilter2' : [3, [18,19,20,21,22,23]]
#           'dropblock' : [30, 100]
          #'device' : device
}

params_no_shuffle = {'batch_size': batch_size,
#          'input_shape': (100, 313, 1),
          'shuffle': False,
          'beta_param': 0.7,
          'mixup': False
          #'device': device
}

TrainDGen_1 = Generator0([features_trn['age'],features_trn['sex'], features_trn['hw'], features_trn['preg'], features_trn['loc'], 
           features_trn['mel1'],features_trn['cqt1'],features_trn['stft1']], 
                        out_lbs_trn,  ## our Y
                        **params)()

#ValDGen_1 = Generator0([features_test[0]['age'],features_test[0]['sex'], features_test[0]['hw'], features_test[0]['preg'], features_test[0]['loc'], 
#           features_test[0]['mel1'],features_test[0]['cqt1'],features_test[0]['stft1']], 
#                        features_test[2],  ## our Y
#                        **params_no_shuffle)()

    
model2.fit(TrainDGen_1,
          validation_data = ([features_test['age'],features_test['sex'], features_test['hw'], 
                              features_test['preg'], features_test['loc'], features_test['mel1'], 
                              features_test['cqt1'], features_test['stft1']], 
                             out_lbs_test), 
                             callbacks=[lr],
                              steps_per_epoch=np.ceil(len(out_lbs_trn)/64),
                             epochs = n_epoch)

Epoch 1/100
40/40 [==============================] - 9s 201ms/step - loss: 0.7671 - accuracy: 0.5098 - auc: 0.5344 - val_loss: 0.7761 - val_accuracy: 0.3994 - val_auc: 0.3535
Epoch 2/100
40/40 [==============================] - 7s 168ms/step - loss: 0.7134 - accuracy: 0.5340 - auc: 0.5511 - val_loss: 0.7773 - val_accuracy: 0.5055 - val_auc: 0.5498
Epoch 3/100
40/40 [==============================] - 7s 174ms/step - loss: 0.7093 - accuracy: 0.4935 - auc: 0.5225 - val_loss: 1.3568 - val_accuracy: 0.5055 - val_auc: 0.5932
Epoch 4/100
40/40 [==============================] - 7s 175ms/step - loss: 0.7084 - accuracy: 0.4987 - auc: 0.5316 - val_loss: 0.8671 - val_accuracy: 0.5198 - val_auc: 0.5866
Epoch 5/100
40/40 [==============================] - 7s 171ms/step - loss: 0.7110 - accuracy: 0.5260 - auc: 0.5452 - val_loss: 0.7184 - val_accuracy: 0.5008 - val_auc: 0.5257
Epoch 6/100
40/40 [==============================] - 7s 173ms/step - loss: 0.6959 - accuracy: 0.5345 - auc: 0.5465 - val_loss

40/40 [==============================] - 7s 175ms/step - loss: 0.6632 - accuracy: 0.5938 - auc: 0.6781 - val_loss: 0.6260 - val_accuracy: 0.6545 - val_auc: 0.7154
Epoch 48/100
40/40 [==============================] - 7s 173ms/step - loss: 0.6735 - accuracy: 0.5809 - auc: 0.6511 - val_loss: 0.6512 - val_accuracy: 0.6450 - val_auc: 0.6632
Epoch 49/100
40/40 [==============================] - 7s 179ms/step - loss: 0.6731 - accuracy: 0.5879 - auc: 0.6656 - val_loss: 0.6522 - val_accuracy: 0.6323 - val_auc: 0.6624
Epoch 50/100
40/40 [==============================] - 7s 178ms/step - loss: 0.6706 - accuracy: 0.5908 - auc: 0.6618 - val_loss: 0.6386 - val_accuracy: 0.6323 - val_auc: 0.7073
Epoch 51/100
40/40 [==============================] - 7s 182ms/step - loss: 0.6758 - accuracy: 0.5818 - auc: 0.6608 - val_loss: 0.6314 - val_accuracy: 0.6751 - val_auc: 0.7251
Epoch 52/100
40/40 [==============================] - 7s 181ms/step - loss: 0.6745 - accuracy: 0.5873 - auc: 0.6492 - val_loss: 0.720

Epoch 94/100
40/40 [==============================] - 7s 177ms/step - loss: 0.6496 - accuracy: 0.6188 - auc: 0.7247 - val_loss: 0.6348 - val_accuracy: 0.6513 - val_auc: 0.6978
Epoch 95/100
40/40 [==============================] - 7s 178ms/step - loss: 0.6524 - accuracy: 0.6164 - auc: 0.7305 - val_loss: 0.6348 - val_accuracy: 0.6545 - val_auc: 0.6972
Epoch 96/100
40/40 [==============================] - 7s 176ms/step - loss: 0.6574 - accuracy: 0.6126 - auc: 0.7051 - val_loss: 0.6325 - val_accuracy: 0.6482 - val_auc: 0.7007
Epoch 97/100
40/40 [==============================] - 7s 182ms/step - loss: 0.6492 - accuracy: 0.6292 - auc: 0.7203 - val_loss: 0.6314 - val_accuracy: 0.6624 - val_auc: 0.7041
Epoch 98/100
40/40 [==============================] - 7s 182ms/step - loss: 0.6614 - accuracy: 0.5923 - auc: 0.6980 - val_loss: 0.6362 - val_accuracy: 0.6593 - val_auc: 0.6977
Epoch 99/100
40/40 [==============================] - 7s 178ms/step - loss: 0.6534 - accuracy: 0.6288 - auc: 0.7209 - va

### 팀코드 수정해야 할 함수들..

In [81]:
params_feature

{'samp_sec': 20,
 'pre_emphasis': 0,
 'hop_length': 256,
 'win_length': 512,
 'n_mels': 100,
 'filter_scale': 1,
 'n_bins': 80,
 'fmin': 10,
 'mel_shape': (100, 313, 1),
 'cqt_shape': (80, 157, 1),
 'stft_shape': (257, 313, 1),
 'model1': 'lcnn1',
 'model2': 'lcnn2',
 'model_fnm1': 'lcnn2/lcnn1_model1.hdf5',
 'model_fnm2': 'lcnn2/lcnn2_model2.hdf5'}

In [82]:
params_feature['mel_shape'] = mel_input_shape
params_feature['cqt_shape'] = cqt_input_shape
params_feature['stft_shape'] = stft_input_shape


In [83]:
import pickle as pk
def save_challenge_model(model_folder, model1, model2, m_name1, m_name2, param_feature) :
    os.makedirs(model_folder, exist_ok=True)
    info_fnm = os.path.join(model_folder, 'desc.pk')
    filename1 = os.path.join(model_folder, m_name1 + '_model1.hdf5')
    filename2 = os.path.join(model_folder, m_name2 + '_model2.hdf5')
    model1.save(filename1)
    model2.save(filename2)
    param_feature['model1'] = m_name1
    param_feature['model2'] = m_name2
    param_feature['model_fnm1'] = filename1
    param_feature['model_fnm2'] = filename2

    with open(info_fnm, 'wb') as f:
        pk.dump(param_feature, f, pk.HIGHEST_PROTOCOL)
    return 1
    

In [84]:
model_folder

'lcnn2'

In [85]:
save_challenge_model(model_folder, model1, model2, m_name1 = 'lcnn1', m_name2 = 'lcnn2', param_feature = params_feature)

1

In [86]:
def load_challenge_model(model_folder, verbose):
    info_fnm = os.path.join(model_folder, 'desc.pk')
    with open(info_fnm, 'rb') as f:
        info_m = pk.load(f)
#    if info_m['model'] == 'toy' :
#        model = get_toy(info_m['mel_shape'])
#    filename = os.path.join(model_folder, info_m['model'] + '_model.hdf5')
#    model.load_weights(filename)
    return info_m

In [87]:
params_feature

{'samp_sec': 20,
 'pre_emphasis': 0,
 'hop_length': 256,
 'win_length': 512,
 'n_mels': 100,
 'filter_scale': 1,
 'n_bins': 80,
 'fmin': 10,
 'mel_shape': (100, 313, 1),
 'cqt_shape': (80, 157, 1),
 'stft_shape': (257, 313, 1),
 'model1': 'lcnn1',
 'model2': 'lcnn2',
 'model_fnm1': 'lcnn2/lcnn1_model1.hdf5',
 'model_fnm2': 'lcnn2/lcnn2_model2.hdf5'}

In [88]:
def run_challenge_model(model, data, recordings, verbose):
    
    murmur_classes = ['Present', 'Unknown', 'Absent']
    outcome_classes = ['Abnormal', 'Normal']
    
    if model['model1'] == 'toy1' :
        model1 = get_toy5_1(model['mel_shape'],model['cqt_shape'],model['stft_shape'] )
    if model['model2'] == 'toy2' :
        model2 = get_toy5_2(model['mel_shape'],model['cqt_shape'],model['stft_shape'])
    if model['model1'] == 'lcnn1' :
        model1 = get_LCNN_1(model['mel_shape'],model['cqt_shape'],model['stft_shape'], use_mel = True, use_cqt = False, use_stft = False)
    if model['model2'] == 'lcnn2' :
        model2 = get_LCNN_2(model['mel_shape'],model['cqt_shape'],model['stft_shape'], use_mel = True, use_cqt = False, use_stft = False)
    model1.load_weights(model['model_fnm1'])
    model2.load_weights(model['model_fnm2'])
    
#    classes = model['classes']
    # Load features.
    features = get_feature_one(data, verbose = 0)

    samp_sec = model['samp_sec'] 
    pre_emphasis = model['pre_emphasis']
    hop_length = model['hop_length']
    win_length = model['win_length']
    n_mels = model['n_mels']
    filter_scale = model['filter_scale']
    n_bins = model['n_bins']
    fmin = model['fmin']
    
    features['mel1'] = []
    for i in range(len(recordings)) :
        mel1 = feature_extract_melspec(recordings[i]/ 32768, samp_sec=samp_sec, pre_emphasis = pre_emphasis, hop_length=hop_length, 
                                           win_length = win_length, n_mels = n_mels)[0]
        features['mel1'].append(mel1)
    M, N = features['mel1'][0].shape
    for i in range(len(features['mel1'])) :
        features['mel1'][i] = features['mel1'][i].reshape(M,N,1)   
    features['mel1'] = np.array(features['mel1'])

    features['cqt1'] = []
    for i in range(len(recordings)) :
        mel1 = feature_extract_cqt(recordings[i]/ 32768, samp_sec=samp_sec, pre_emphasis = pre_emphasis, filter_scale = filter_scale, 
                                        n_bins = n_bins, fmin = fmin)[0]
        features['cqt1'].append(mel1)
    M, N = features['cqt1'][0].shape
    for i in range(len(features['cqt1'])) :
        features['cqt1'][i] = features['cqt1'][i].reshape(M,N,1)   
    features['cqt1'] = np.array(features['cqt1'])

    features['stft1'] = []
    for i in range(len(recordings)) :
        mel1 = feature_extract_stft(recordings[i]/ 32768, samp_sec=samp_sec, pre_emphasis = pre_emphasis, hop_length=hop_length, 
                                       win_length = win_length)[0]
        features['stft1'].append(mel1)
    M, N = features['stft1'][0].shape
    for i in range(len(features['stft1'])) :
        features['stft1'][i] = features['stft1'][i].reshape(M,N,1)           
    features['stft1'] = np.array(features['stft1'])

    #    print(features)
    # Impute missing data.
    res1 = model1.predict([features['age'], features['sex'], features['hw'], features['preg'], features['loc'], features['mel1'], features['cqt1'], features['stft1']])
    res2 = model2.predict([features['age'], features['sex'], features['hw'], features['preg'], features['loc'], features['mel1'], features['cqt1'], features['stft1']])

    # Get classifier probabilities.
    idx1 = res1.argmax(axis=0)[0]
    murmur_probabilities = res1[idx1,]  ## mumur 확률 최대화 되는 애 뽑기
    outcome_probabilities = res2.mean(axis = 0) ##  outcome 은 그냥 평균으로 뽑기
#    idx = np.argmax(prob1)

    ## 이부분도 생각 필요.. rule 을 cost를 maximize 하는 기준으로 threshold 탐색 필요할지도..
    # Choose label with highest probability.
    murmur_labels = np.zeros(len(murmur_classes), dtype=np.int_)
    idx = np.argmax(murmur_probabilities)
    murmur_labels[idx] = 1
    outcome_labels = np.zeros(len(outcome_classes), dtype=np.int_)
    idx = np.argmax(outcome_probabilities)
    outcome_labels[idx] = 1
    
    # Concatenate classes, labels, and probabilities.
    classes = murmur_classes + outcome_classes
    labels = np.concatenate((murmur_labels, outcome_labels))
    probabilities = np.concatenate((murmur_probabilities, outcome_probabilities))
    
    return classes, labels, probabilities

### 대회 평가용 run_model 함수 (수정 불필요)

In [89]:
# Run model.
def run_model(model_folder, data_folder, output_folder, allow_failures, verbose):
    # Load models.
    if verbose >= 1:
        print('Loading Challenge model...')

    model = load_challenge_model(model_folder, verbose) ### Teams: Implement this function!!!

    # Find the patient data files.
    patient_files = find_patient_files(data_folder)
    num_patient_files = len(patient_files)

    if num_patient_files==0:
        raise Exception('No data was provided.')

    # Create a folder for the Challenge outputs if it does not already exist.
    os.makedirs(output_folder, exist_ok=True)

    # Run the team's model on the Challenge data.
    if verbose >= 1:
        print('Running model on Challenge data...')

    # Iterate over the patient files.
    for i in range(num_patient_files):
        if verbose >= 2:
            print('    {}/{}...'.format(i+1, num_patient_files))

        patient_data = load_patient_data(patient_files[i])
        recordings = load_recordings(data_folder, patient_data)

        # Allow or disallow the model to fail on parts of the data; helpful for debugging.
        try:
            classes, labels, probabilities = run_challenge_model(model, patient_data, recordings, verbose) ### Teams: Implement this function!!!
        except:
            if allow_failures:
                if verbose >= 2:
                    print('... failed.')
                classes, labels, probabilities = list(), list(), list()
            else:
                raise

        # Save Challenge outputs.
        head, tail = os.path.split(patient_files[i])
        root, extension = os.path.splitext(tail)
        output_file = os.path.join(output_folder, root + '.csv')
        patient_id = get_patient_id(patient_data)
        save_challenge_outputs(output_file, patient_id, classes, labels, probabilities)

    if verbose >= 1:
        print('Done.')

In [90]:
## 우리 모형 저장된 폴더이름
model_folder

'lcnn2'

In [91]:
## 테스트 데이터 폴더
test_folder

'/home/ubuntu/data/hmd/murmur/test'

In [92]:
## 테스트 데이터에 모형 돌려서 스코어 저장할 폴더
output_folder = '/home/ubuntu/hmd/notebooks/out_lcnn2'

In [93]:
run_model(model_folder, test_folder, output_folder, allow_failures = True, verbose = 1)

Loading Challenge model...
Running model on Challenge data...


Done.


In [94]:
!cat /home/ubuntu/hmd/notebooks/out_lcnn2/49628.csv

#49628
Present,Unknown,Absent,Abnormal,Normal
1,0,0,1,0
0.6172488,0.0019498507,0.38080138,0.6646719,0.3353281


In [95]:
from evaluate_model import *

In [96]:
murmur_scores, outcome_scores = evaluate_model(test_folder, output_folder)
classes, auroc, auprc, auroc_classes, auprc_classes, f_measure, f_measure_classes, accuracy, accuracy_classes, weighted_accuracy, cost = murmur_scores
murmur_output_string = 'AUROC,AUPRC,F-measure,Accuracy,Weighted Accuracy,Cost\n{:.3f},{:.3f},{:.3f},{:.3f},{:.3f},{:.3f}\n'.format(auroc, auprc, f_measure, accuracy, weighted_accuracy, cost)
murmur_class_output_string = 'Classes,{}\nAUROC,{}\nAUPRC,{}\nF-measure,{}\nAccuracy,{}\n'.format(
    ','.join(classes),
    ','.join('{:.3f}'.format(x) for x in auroc_classes),
    ','.join('{:.3f}'.format(x) for x in auprc_classes),
    ','.join('{:.3f}'.format(x) for x in f_measure_classes),
    ','.join('{:.3f}'.format(x) for x in accuracy_classes))

classes, auroc, auprc, auroc_classes, auprc_classes, f_measure, f_measure_classes, accuracy, accuracy_classes, weighted_accuracy, cost = outcome_scores
outcome_output_string = 'AUROC,AUPRC,F-measure,Accuracy,Weighted Accuracy,Cost\n{:.3f},{:.3f},{:.3f},{:.3f},{:.3f},{:.3f}\n'.format(auroc, auprc, f_measure, accuracy, weighted_accuracy, cost)
outcome_class_output_string = 'Classes,{}\nAUROC,{}\nAUPRC,{}\nF-measure,{}\nAccuracy,{}\n'.format(
    ','.join(classes),
    ','.join('{:.3f}'.format(x) for x in auroc_classes),
    ','.join('{:.3f}'.format(x) for x in auprc_classes),
    ','.join('{:.3f}'.format(x) for x in f_measure_classes),
    ','.join('{:.3f}'.format(x) for x in accuracy_classes))

output_string = '#Murmur scores\n' + murmur_output_string + '\n#Outcome scores\n' + outcome_output_string \
    + '\n#Murmur scores (per class)\n' + murmur_class_output_string + '\n#Outcome scores (per class)\n' + outcome_class_output_string

if len(sys.argv) == 3:
    print(output_string)
elif len(sys.argv) == 4:
    with open(sys.argv[3], 'w') as f:
        f.write(output_string)

#Murmur scores
AUROC,AUPRC,F-measure,Accuracy,Weighted Accuracy,Cost
0.803,0.657,0.516,0.785,0.792,14695.035

#Outcome scores
AUROC,AUPRC,F-measure,Accuracy,Weighted Accuracy,Cost
0.720,0.717,0.666,0.670,0.583,14715.841

#Murmur scores (per class)
Classes,Present,Unknown,Absent
AUROC,0.955,0.574,0.881
AUPRC,0.869,0.168,0.934
F-measure,0.679,0.000,0.870
Accuracy,0.947,0.000,0.820

#Outcome scores (per class)
Classes,Abnormal,Normal
AUROC,0.720,0.720
AUPRC,0.772,0.663
F-measure,0.627,0.704
Accuracy,0.541,0.806



### Threshold 변경해가며 결과 탐색

In [97]:
label_folder = test_folder
murmur_classes = ['Present', 'Unknown', 'Absent']
outcome_classes = ['Abnormal', 'Normal']

# Load and parse label and model output files.
label_files, output_files = find_challenge_files(label_folder, output_folder)
murmur_labels = load_murmurs(label_files, murmur_classes)
murmur_binary_outputs, murmur_scalar_outputs = load_classifier_outputs(output_files, murmur_classes)
outcome_labels = load_outcomes(label_files, outcome_classes)
outcome_binary_outputs, outcome_scalar_outputs = load_classifier_outputs(output_files, outcome_classes)


print(np.mean(murmur_scalar_outputs[:,0]))
print(np.mean(murmur_scalar_outputs[:,2]))
print(np.mean(outcome_scalar_outputs[:,0]))
print(np.mean(outcome_scalar_outputs[:,1]))

0.46400997204188477
0.5167402682722513
0.4966289968062827
0.503371002774869


In [98]:
## threshold 바꿔가면서 결과 출력

for th1 in [0.01, 0.05, 0.1, 0.15,0.2, 0.25, 0.3, 0.35, 0.4, 0.45, 0.5, 0.55, 0.6, 0.65, 0.7, 0.75, 0.8] :
    murmur_binary_outputs[:,0] = murmur_scalar_outputs[:,0] > th1
    murmur_binary_outputs[:,2] = murmur_scalar_outputs[:,2] > 1 - th1
    outcome_binary_outputs[:,0] = outcome_scalar_outputs[:,0] > th1
    outcome_binary_outputs[:,1] = outcome_scalar_outputs[:,1] > 1 - th1
    # For each patient, set the 'Present' or 'Abnormal' class to positive if no class is positive or if multiple classes are positive.
    murmur_labels = enforce_positives(murmur_labels, murmur_classes, 'Present')
    murmur_binary_outputs = enforce_positives(murmur_binary_outputs, murmur_classes, 'Present')
    outcome_labels = enforce_positives(outcome_labels, outcome_classes, 'Abnormal')
    outcome_binary_outputs = enforce_positives(outcome_binary_outputs, outcome_classes, 'Abnormal')
    # Evaluate the murmur model by comparing the labels and model outputs.
    murmur_auroc, murmur_auprc, murmur_auroc_classes, murmur_auprc_classes = compute_auc(murmur_labels, murmur_scalar_outputs)
    murmur_f_measure, murmur_f_measure_classes = compute_f_measure(murmur_labels, murmur_binary_outputs)
    murmur_accuracy, murmur_accuracy_classes = compute_accuracy(murmur_labels, murmur_binary_outputs)
    murmur_weighted_accuracy = compute_weighted_accuracy(murmur_labels, murmur_binary_outputs, murmur_classes) # This is the murmur scoring metric.
    murmur_cost = compute_cost(outcome_labels, murmur_binary_outputs, outcome_classes, murmur_classes) # Use *outcomes* to score *murmurs* for the Challenge cost metric, but this is not the actual murmur scoring metric.
    murmur_scores = (murmur_classes, murmur_auroc, murmur_auprc, murmur_auroc_classes, murmur_auprc_classes, \
                 murmur_f_measure, murmur_f_measure_classes, murmur_accuracy, murmur_accuracy_classes, murmur_weighted_accuracy, murmur_cost)

    # Evaluate the outcome model by comparing the labels and model outputs.
    outcome_auroc, outcome_auprc, outcome_auroc_classes, outcome_auprc_classes = compute_auc(outcome_labels, outcome_scalar_outputs)
    outcome_f_measure, outcome_f_measure_classes = compute_f_measure(outcome_labels, outcome_binary_outputs)
    outcome_accuracy, outcome_accuracy_classes = compute_accuracy(outcome_labels, outcome_binary_outputs)
    outcome_weighted_accuracy = compute_weighted_accuracy(outcome_labels, outcome_binary_outputs, outcome_classes)
    outcome_cost = compute_cost(outcome_labels, outcome_binary_outputs, outcome_classes, outcome_classes) # This is the clinical outcomes scoring metric.
    outcome_scores = (outcome_classes, outcome_auroc, outcome_auprc, outcome_auroc_classes, outcome_auprc_classes, \
                  outcome_f_measure, outcome_f_measure_classes, outcome_accuracy, outcome_accuracy_classes, outcome_weighted_accuracy, outcome_cost)



    classes, auroc, auprc, auroc_classes, auprc_classes, f_measure, f_measure_classes, accuracy, accuracy_classes, weighted_accuracy, cost = murmur_scores
    murmur_output_string = 'AUROC,AUPRC,F-measure,Accuracy,Weighted Accuracy,Cost\n{:.3f},{:.3f},{:.3f},{:.3f},{:.3f},{:.3f}\n'.format(auroc, auprc, f_measure, accuracy, weighted_accuracy, cost)
    murmur_class_output_string = 'Classes,{}\nAUROC,{}\nAUPRC,{}\nF-measure,{}\nAccuracy,{}\n'.format(
    ','.join(classes),
    ','.join('{:.3f}'.format(x) for x in auroc_classes),
    ','.join('{:.3f}'.format(x) for x in auprc_classes),
    ','.join('{:.3f}'.format(x) for x in f_measure_classes),
    ','.join('{:.3f}'.format(x) for x in accuracy_classes))

    classes, auroc, auprc, auroc_classes, auprc_classes, f_measure, f_measure_classes, accuracy, accuracy_classes, weighted_accuracy, cost = outcome_scores
    outcome_output_string = 'AUROC,AUPRC,F-measure,Accuracy,Weighted Accuracy,Cost\n{:.3f},{:.3f},{:.3f},{:.3f},{:.3f},{:.3f}\n'.format(auroc, auprc, f_measure, accuracy, weighted_accuracy, cost)
    outcome_class_output_string = 'Classes,{}\nAUROC,{}\nAUPRC,{}\nF-measure,{}\nAccuracy,{}\n'.format(
    ','.join(classes),
    ','.join('{:.3f}'.format(x) for x in auroc_classes),
    ','.join('{:.3f}'.format(x) for x in auprc_classes),
    ','.join('{:.3f}'.format(x) for x in f_measure_classes),
    ','.join('{:.3f}'.format(x) for x in accuracy_classes))

    output_string = '#Murmur scores\n' + murmur_output_string + '\n#Outcome scores\n' + outcome_output_string \
                + '\n#Murmur scores (per class)\n' + murmur_class_output_string + '\n#Outcome scores (per class)\n' + outcome_class_output_string
    print("threshold: ", th1)
    print(output_string)
    print("-------------")


threshold:  0.01
#Murmur scores
AUROC,AUPRC,F-measure,Accuracy,Weighted Accuracy,Cost
0.803,0.657,0.111,0.199,0.512,15140.890

#Outcome scores
AUROC,AUPRC,F-measure,Accuracy,Weighted Accuracy,Cost
0.720,0.717,0.339,0.513,0.840,15140.890

#Murmur scores (per class)
Classes,Present,Unknown,Absent
AUROC,0.955,0.574,0.881
AUPRC,0.869,0.168,0.934
F-measure,0.332,0.000,0.000
Accuracy,1.000,0.000,0.000

#Outcome scores (per class)
Classes,Abnormal,Normal
AUROC,0.720,0.720
AUPRC,0.772,0.663
F-measure,0.678,0.000
Accuracy,1.000,0.000

-------------
threshold:  0.05
#Murmur scores
AUROC,AUPRC,F-measure,Accuracy,Weighted Accuracy,Cost
0.803,0.657,0.111,0.199,0.512,15140.890

#Outcome scores
AUROC,AUPRC,F-measure,Accuracy,Weighted Accuracy,Cost
0.720,0.717,0.339,0.513,0.840,15140.890

#Murmur scores (per class)
Classes,Present,Unknown,Absent
AUROC,0.955,0.574,0.881
AUPRC,0.869,0.168,0.934
F-measure,0.332,0.000,0.000
Accuracy,1.000,0.000,0.000

#Outcome scores (per class)
Classes,Abnormal,Normal
AU